-Google Colab Code

In [ ]:
!nvidia-smi 

 

Thu Dec 25 09:47:46 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   32C    P8              9W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [ ]:
!pip install -q supervision "ultralytics<=8.3.40" 
 

In [ ]:
 
!yolo settings sync=False

JSONDict("/root/.config/Ultralytics/settings.json"):
{
  "settings_version": "0.0.6",
  "datasets_dir": "/content/datasets",
  "weights_dir": "weights",
  "runs_dir": "runs",
  "uuid": "569f3ba64b326db489132663f79cd37279811de477381b83ac131e6cdd129cbb",
  "sync": false,
  "api_key": "",
  "openai_api_key": "",
  "clearml": true,
  "comet": true,
  "dvc": true,
  "hub": true,
  "mlflow": true,
  "neptune": true,
  "raytune": true,
  "tensorboard": true,
  "wandb": false,
  "vscode_msg": true
}
💡 Learn more about Ultralytics Settings at https://docs.ultralytics.com/quickstart/#ultralytics-settings


In [ ]:
import cv2
import numpy as np
import supervision as sv
from tqdm import tqdm
from ultralytics import YOLO
from supervision.assets import VideoAssets, download_assets
from collections import defaultdict, deque

download_assets(VideoAssets.VEHICLES)
 

vehicles.mp4 asset download complete. 



'vehicles.mp4'

In [22]:
INPUT_VIDEO = "vehicles.mp4"
OUTPUT_VIDEO = "vehicles_output.mp4"
MODEL = "yolov8m.pt"
CONFIDENCE_THRESHOLD = 0.3
IOU_THRESHOLD = 0.5
MODEL_RESOLUTION = 1280

In [ ]:
 
box_coordinates = np.array([
    [1252, 787],     
    [2298, 803],    
    [5039, 2159],    
    [-550, 2159]     
])

 
box_width_metres = 25       
box_height_metres = 250    

 
box = np.array([
    [0, 0],
    [box_width_metres - 1, 0],
    [box_width_metres - 1, box_height_metres - 1],
    [0, box_height_metres - 1],
])


In [ ]:
from google.colab.patches import cv2_imshow

 
cap = cv2.VideoCapture(INPUT_VIDEO)

 
while cap.isOpened():

    success, frame = cap.read()
    if not success:
        break

 
    annotated_frame = frame.copy()

 
    annotated_frame = sv.draw_polygon(
        scene=annotated_frame,
        polygon=box_coordinates,
        color=sv.Color.RED,
        thickness=4
    )

  
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
 
cap.release()

 


In [ ]:

import cv2
import numpy as np


homography = cv2.getPerspectiveTransform(
    box_coordinates .astype(np.float32),
    box.astype(np.float32)
)

 
def transform_points(points):
    if points.size == 0:
        return points
    pts = points.reshape(-1, 1, 2).astype(np.float32)
    warped = cv2.perspectiveTransform(pts, homography)
    return warped.reshape(-1, 2)



In [ ]:

model = YOLO(MODEL)

video_info = sv.VideoInfo.from_video_path(INPUT_VIDEO)


frame_generator = sv.get_video_frames_generator(INPUT_VIDEO)

 
byte_track = sv.ByteTrack(
    frame_rate=video_info.fps,
    track_activation_threshold=CONFIDENCE_THRESHOLD
)

 
thickness = sv.calculate_optimal_line_thickness(
    resolution_wh=video_info.resolution_wh
)


text_scale = sv.calculate_optimal_text_scale(
    resolution_wh=video_info.resolution_wh
)

bbox_annot = sv.BoxAnnotator(thickness=thickness)


label_annot = sv.LabelAnnotator(
    text_scale=text_scale,
    text_thickness=thickness,
    text_position=sv.Position.BOTTOM_CENTER
)


trace_annot = sv.TraceAnnotator(
    thickness=thickness,
    trace_length=video_info.fps * 2,
    position=sv.Position.BOTTOM_CENTER
)


polygon_zone = sv.PolygonZone(
    polygon=box_coordinates
)


coordinates = defaultdict(lambda: deque(maxlen=video_info.fps))


In [28]:
VEHICLE_CLASSES = [2, 3, 5, 7]
# car, motorcycle, bus, truck


In [ ]:
 
fourcc = cv2.VideoWriter_fourcc(*"mp4v")

 
video_writer = cv2.VideoWriter(
    OUTPUT_VIDEO,
    fourcc,
    video_info.fps,
    video_info.resolution_wh
)



In [ ]:
 
vehicle_count = set()

def calculate_speed(track_id, world_point):
 

 
    coords = coordinates[track_id]
    coords.append(world_point)

 
    if len(coords) < 2:
        return None


    distance_m = np.linalg.norm(coords[-1] - coords[0])
    time_s = len(coords) / video_info.fps

 
    speed_mps = distance_m / time_s
    speed_kmh = speed_mps * 3.6

    return speed_kmh



In [ ]:
for frame in tqdm(frame_generator, total=video_info.total_frames):

 
    results = model(
        frame,
        imgsz=MODEL_RESOLUTION,
        conf=CONFIDENCE_THRESHOLD,
        iou=IOU_THRESHOLD,
        verbose=False
    )[0]

    
    detections = sv.Detections.from_ultralytics(results)

 
    mask = np.isin(detections.class_id, VEHICLE_CLASSES)
    detections = detections[mask]
    detections = detections[polygon_zone.trigger(detections)]

    detections = byte_track.update_with_detections(detections)

    if len(detections) == 0:
        video_writer.write(frame)
        continue

    points = detections.get_anchors_coordinates(
        anchor=sv.Position.BOTTOM_CENTER
    )

 
    world_points = transform_points(points)
 
    labels = []
    for tracker_id, world_pt in zip(detections.tracker_id, world_points):

        speed = calculate_speed(tracker_id, world_pt)

      
        vehicle_count.add(tracker_id)

        if speed is None:
            labels.append(f"ID {tracker_id}")
        else:
            labels.append(f"ID {tracker_id} | {int(speed)} km/h")
 
    annotated = frame.copy()
 
    annotated = sv.draw_polygon(
        annotated,
        box_coordinates,
        color=sv.Color.RED,
        thickness=2
    )

 
    annotated = bbox_annot.annotate(annotated, detections)
    annotated = trace_annot.annotate(annotated, detections)
    annotated = label_annot.annotate(annotated, detections, labels)
 
    text = f"Vehicles counted: {len(vehicle_count)}"
    font = cv2.FONT_HERSHEY_SIMPLEX

    font_scale = video_info.resolution_wh[0] / 1200
    thickness = 2

    (text_width, text_height), _ = cv2.getTextSize(
        text, font, font_scale, thickness
    )

    x = (video_info.resolution_wh[0] - text_width) // 2
    y = text_height + 20

    pad_x = 20
    pad_y = 15

    overlay = annotated.copy()

    cv2.rectangle(
        overlay,
        (x - pad_x, y - text_height - pad_y),
        (x + text_width + pad_x, y + pad_y),
        (40, 40, 40),
        -1
    )

    cv2.addWeighted(overlay, 0.6, annotated, 0.4, 0, annotated)

    cv2.putText(
        annotated,
        text,
        (x, y),
        font,
        font_scale,
        (255, 255, 255),
        thickness,
        cv2.LINE_AA
    )
 
    video_writer.write(annotated)

 
video_writer.release()



100%|██████████| 538/538 [01:25<00:00,  6.29it/s]
